In [1]:
import logging
import sys 
from io import StringIO
import datetime

class LevelFilter(logging.Filter):
    ##example: yFilter1=LevelFilter((logging.INFO, logging.WARNING, logging.ERROR))
    def __init__(self, levels):
        self.levels = levels

    def filter(self, record):
        return record.levelno in self.levels



def setLogging(Filter):
    ylog_stream=StringIO()

    logging.basicConfig(level=logging.DEBUG)
    yLog=logging.getLogger()
   # yLog.handlers.pop()
   # yLog.propagate=False   #when no handler to handle a log message at current level
    #yLog.setLevel(logging.DEBUG)  # critial,error,warning,info,debug, notset 
   # logging.basicConfig(format='%(asctime)s | %(levelname)s: %(message)s', level=logging.NOTSET)

    yHandler=logging.StreamHandler(sys.stdout) #stdout, so cell output is white   
   
    log_format=' %(funcName)s%(lineno)s|%(levelno)s: %(message)s [%(filename)s%(asctime)s]'
    dt_format= '%m/%d:%I:%M'#  %p'
    yHandler.setFormatter(logging.Formatter(log_format, datefmt=dt_format))
    
    

    yHandler1=logging.StreamHandler(ylog_stream) #stderr, so cell output is red   
    
    log_format=' %(funcName)s%(lineno)s|%(levelname)s: %(message)s [%(filename)s%(asctime)s]'
    dt_format= '%m/%d:%I:%M'#  %p'
    yHandler1.setFormatter(logging.Formatter(log_format, datefmt=dt_format))   

    yHandler1.setLevel=(logging.ERROR)
    yHandler1.addFilter(Filter)
    
    yHandler.setLevel=(logging.CRITICAL)


    if (yLog.hasHandlers()):
        yLog.handlers.clear()  #c logging has its default handle if logging basic config is called 
    yLog.addHandler(yHandler)
    yLog.addHandler(yHandler1)
    
   
    lgd=lambda y: yLog.debug(y)
    lgi=lambda y: yLog.info(y)
    lgw=lambda y: yLog.warning(y)
    lge=lambda y: yLog.error(y)
    lgc=lambda y: yLog.critical(y)

    lgi('logging started at:' + datetime.datetime.now().strftime('%x'))

    return lgc,lge,lgw,lgi,lgd,yLog, ylog_stream




In [2]:
yFilter1=LevelFilter((logging.INFO, logging.WARNING, logging.ERROR))
lgc,lge,lgw,lgi,lgd, yLog, yLogStrm=setLogging(yFilter1)


lgc("testing lg.criticalsdfffffffffff    ")
lgd("testing lg.debug")
lgw("testing lg.warning")
lgi("testing lg.info")
print ('-----------log stream')
print (yLogStrm.getvalue())



 <lambda>52|20: logging started at:04/22/21 [<ipython-input-1-f41ca5364651>04/22:11:22]
 <lambda>55|50: testing lg.criticalsdfffffffffff     [<ipython-input-1-f41ca5364651>04/22:11:22]
 <lambda>51|10: testing lg.debug [<ipython-input-1-f41ca5364651>04/22:11:22]
 <lambda>53|30: testing lg.warning [<ipython-input-1-f41ca5364651>04/22:11:22]
 <lambda>52|20: testing lg.info [<ipython-input-1-f41ca5364651>04/22:11:22]
-----------log stream
 <lambda>52|INFO: logging started at:04/22/21 [<ipython-input-1-f41ca5364651>04/22:11:22]
 <lambda>53|WARNING: testing lg.warning [<ipython-input-1-f41ca5364651>04/22:11:22]
 <lambda>52|INFO: testing lg.info [<ipython-input-1-f41ca5364651>04/22:11:22]



In [6]:
lgw("testing lg.warning2")
lgw("testing lg.warning3")

 <lambda>53|30: testing lg.warning2 [<ipython-input-1-194729e946d9>04/22:10:57]
 <lambda>53|30: testing lg.warning3 [<ipython-input-1-194729e946d9>04/22:10:57]


In [7]:
lgw("testing lg.warning4")
print('asd')

 <lambda>53|30: testing lg.warning4 [<ipython-input-1-194729e946d9>04/22:10:57]
asd


In [9]:
# "C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\Automating Stock Investing Technical Analysis With Python _ by Farhad Malik _ FinTechExplained _ Medium_btc.pdf"
# for TA in python https://technical-analysis-library-in-python.readthedocs.io/en/latest/
from ta.momentum import RSIIndicator
from ta.trend import MACD
import numpy as np
import pandas
from ta.volatility import BollingerBands
import talib
import a_utils

class Company:
    def __init__(self, symbol):
        self.symbol = symbol
        self.technical_indicators = None    ### pd dataframe 
        self.prices = None  ### pd series 
        self.img_path=r'C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\historical_data\a_Debug'

    # you don;t need the "y" if this function is declared outside of the Company class
    # this is an instance method, requires "self" as the first arugment in a instance method
    def generate_buy_sell_signals(y, condition_buy, condition_sell, dataframe, strategy):
        last_signal = None
        indicators = []
        buy = []
        sell = [] 

        
        #print(" Buy sell signal() condition_buy type=", type(condition_sell))
        #print(" Buy sell signal() dataframe type=", type(dataframe))
        #print(" Buy sell signal() buy type=", type(buy))
        #print("genbuysellsignal():company.ti ?DF shape", dataframe.shape)

        for i in range(0, len(dataframe)):
            # if buy condition is true and last signal was not Buy
            if condition_buy(i, dataframe) and last_signal != 'Buy':
                last_signal = 'Buy'
                indicators.append(last_signal)
                buy.append(dataframe['Close'].iloc[i])
                sell.append(np.nan)
            # if sell condition is true and last signal was Buy
            elif condition_sell(i, dataframe)  and last_signal == 'Buy':
                last_signal = 'Sell'
                indicators.append(last_signal)
                buy.append(np.nan)
                sell.append(dataframe['Close'].iloc[i])
            else:
                indicators.append(last_signal)
                buy.append(np.nan)
                sell.append(np.nan)

        dataframe[f'{strategy}_Last_Signal'] = np.array(last_signal)
        dataframe[f'{strategy}_Indicator'] = np.array(indicators)
        dataframe[f'{strategy}_Buy'] = np.array(buy)
        dataframe[f'{strategy}_Sell'] = np.array(sell)
        # print("genbuysellsignal():company.ti shape", company.technical_indicators.shape)
        # print("genbuysellsignal():company.ti ?DF shape", dataframe.shape)
def get_sma(config, company):
        close_prices = company.prices
    
        DF2=company.technical_indicators  

        print("get_sma():df2 shape", DF2.shape)
        
        SMAPeriod=10

        ySMA=talib.SMA(close_prices, timeperiod=SMAPeriod)
        DF2['SMA']=ySMA

        print("get_sma():df2 shape", DF2.shape)
        print("get_sma():df2 type", type(DF2))
        print("get_sma():company.ti shape, before gen_signal", company.technical_indicators.shape)
        company.generate_buy_sell_signals(lambda x, dataframe: DF2['SMA'].values[x] < DF2['Close'].iloc[x] , 
                                          lambda x, dataframe: DF2['SMA'].values[x] > DF2['Close'].iloc[x], DF2, 'SMA')
        
        #print("genbuysellsignal():company.ti shape after gen_signal", company.technical_indicators.shape)
        return DF2         

def set_technical_indicators(config, company):
        company.technical_indicators = pandas.DataFrame()
        company.technical_indicators['Close'] = company.prices  # tech_indicator is now fully stacked with on column , with index

        #print ("new", company.technical_indicators)
        
        get_sma(config, company)
        #print ("after SMA",company.technical_indicators)

        #print ("before MACD",yGoog.technical_indicators)
        get_macd(config, company)
        
        #print ("before RSI",yGoog.technical_indicators)
        get_rsi(config, company)
        
        #print ("After RSI", company.technical_indicators)
        get_bollinger_bands(config, company)

def get_macd(config, company):
        close_prices = company.prices
        dataframe1 = company.technical_indicators
        window_slow = 26
        signal = 9
        window_fast = 12
        macd = MACD(company.prices, window_slow, window_fast, signal)
        dataframe1['MACD'] = macd.macd()
        dataframe1['MACD_Histogram'] = macd.macd_diff()
        dataframe1['MACD_Signal'] = macd.macd_signal()

        #print("df len", len(dataframe1))
    
        company.generate_buy_sell_signals(lambda x, dataframe: dataframe['MACD'].values[x] < dataframe['MACD_Signal'].iloc[x] , 
                                        lambda x, dataframe: dataframe['MACD'].values[x] > dataframe['MACD_Signal'].iloc[x], 
                                        dataframe1, 'MACD')
                                        
        return dataframe1

def get_rsi(config, company):
    close_prices = company.prices
    dataframe = company.technical_indicators
    rsi_time_period = 20
    rsi_indicator = RSIIndicator(close_prices, rsi_time_period)
    dataframe['RSI'] = rsi_indicator.rsi()
    low_rsi = 40
    high_rsi = 70
    company.generate_buy_sell_signals(
        lambda x, dataframe: dataframe['RSI'].values[x] < low_rsi,
        lambda x, dataframe: dataframe['RSI'].values[x] > high_rsi, dataframe, 'RSI')
    return dataframe


def get_bollinger_bands(config, company):
    close_prices = company.prices
    dataframe = company.technical_indicators
    window = 20
    indicator_bb = BollingerBands(close=close_prices, window=window,window_dev=2)
    # Add Bollinger Bands features
    dataframe['Bollinger_Bands_Middle'] =indicator_bb.bollinger_mavg()
    dataframe['Bollinger_Bands_Upper'] =indicator_bb.bollinger_hband()
    dataframe['Bollinger_Bands_Lower'] =indicator_bb.bollinger_lband()
    company.generate_buy_sell_signals(
        lambda x, signal: signal['Close'].values[x] < signal['Bollinger_Bands_Lower'].values[x], 
        lambda x, signal: signal['Close'].values[x] >
        signal['Bollinger_Bands_Upper'].values[x],
        dataframe, 'Bollinger_Bands')
    return dataframe

In [10]:
# plotting 
# from "C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\Automating Stock Investing Technical Analysis With Python _ by Farhad Malik _ FinTechExplained _ Medium_btc.pdf"
# started wit matplotlib tutorial:
# https://matplotlib.org/stable/tutorials/index.html


import os
from matplotlib import pyplot as plt
import numpy as np
class TechnicalIndicatorsChartPlotter:
    def plot_price_and_signals(self, fig, company, data, strategy,axs):
            last_signal_val = data[f'{strategy}_Last_Signal'].values[-1]
            last_signal = 'Unknown' if not last_signal_val else last_signal_val
        
            title = f'Close Price Buy/Sell Signals using {strategy}.Last Signal: {last_signal}'
            fig.suptitle(f'Top: {company.symbol} Stock Price. Bottom:{strategy}')

            if not data[f'{strategy}_Buy'].isnull().all():
                axs[0].scatter(data.index, data[f'{strategy}_Buy'], color='green', label='Buy Signal', marker='^', alpha=1)

            if not data[f'{strategy}_Sell'].isnull().all():
                axs[0].scatter(data.index, data[f'{strategy}_Sell'], color='red', label='Sell Signal', marker='v', alpha=1)
                axs[0].plot(company.prices, label='Close Price',color='blue', alpha=0.35)

            plt.xticks(rotation=45)
            axs[0].set_title(title)
            axs[0].set_xlabel('Date', fontsize=18)
            axs[0].set_ylabel('Close Price', fontsize=18)
            axs[0].legend(loc='upper left')
            axs[0].grid()

    def plot_macd(self, company):
            image = f'images/{company.symbol}_macd.png'
            macd = company.technical_indicators
            # Create and plot the graph
            fig, axs = plt.subplots(2, sharex=True, figsize=(13,9))
            self.plot_price_and_signals(fig, company, macd, 'MACD', axs)
            axs[1].plot(macd['MACD'], label=company.symbol+' MACD', color= 'green')
            axs[1].plot(macd['MACD_Signal'], label='Signal Line',color='orange')
            positive = macd['MACD_Histogram'][(macd['MACD_Histogram'] >=0)]
            negative = macd['MACD_Histogram'][(macd['MACD_Histogram'] <0)]
            axs[1].bar(positive.index, positive, color='green')
            axs[1].bar(negative.index, negative, color='red')
            axs[1].legend(loc='upper left')
            axs[1].grid()
            #print(os.path.abspath(image))
            
            self.save_plot(company,'macd', plt)
            plt.show()

    def plot_rsi(self, company):
            image = f'images/{company.symbol}_rsi.png'
            rsi = company.technical_indicators
            low_rsi = 40
            high_rsi = 70
        #plt.style.use('default')
            fig, axs = plt.subplots(2, sharex=True, figsize=(13, 9))
            self.plot_price_and_signals(fig, company, rsi, 'RSI', axs)
            axs[1].fill_between(rsi.index, y1=low_rsi, y2=high_rsi, color='#adccff', alpha=0.3)
            axs[1].plot(rsi['RSI'], label='RSI', color='blue',alpha=0.35)
            axs[1].legend(loc='upper left')
            axs[1].grid()
           
            self.save_plot(company,'rsi', plt)  
            plt.show()

    def plot_bollinger_bands(self, company):
            image = f'images/{company.symbol}_bb.png'
            bollinger_bands = company.technical_indicators

            
            fig, axs = plt.subplots(2, sharex=True, figsize=(13, 9))

            self.plot_price_and_signals(fig, company, bollinger_bands, 'Bollinger_Bands', axs)

            axs[1].plot(bollinger_bands['Bollinger_Bands_Middle'], label='Middle', color='blue', alpha=0.35)
            axs[1].plot(bollinger_bands['Bollinger_Bands_Upper'], label='Upper', color='green', alpha=0.35)
            axs[1].plot(bollinger_bands['Bollinger_Bands_Lower'], label='Lower', color='red', alpha=0.35)
            axs[1].fill_between(bollinger_bands.index, bollinger_bands['Bollinger_Bands_Lower'], bollinger_bands['Bollinger_Bands_Upper'], alpha=0.1)
            axs[1].legend(loc='upper left')

            axs[1].grid()
            
            self.save_plot(company,'BB', plt)  
            plt.show()
            
    def plot_sma(self, company):
            
            sma = company.technical_indicators
            # Create and plot the graph
            fig, axs = plt.subplots(2, sharex=True, figsize=(13,9))
            ##### self.plot_price_and_signals(fig, company, sma, 'SMA', axs)
            axs[1].plot(sma['SMA'],  label=company.symbol+' SMA', color= 'green')
            axs[1].plot(sma['Close'], label='Signal Line',color='orange')
            
    
            axs[1].legend(loc='upper left')
            axs[1].grid()
           
                 
            # save image before the show
            self.save_plot(company, 'SMA', plt )
            plt.show()



    def save_plot(self, company, strategy, plot):
            filenm = f'{company.symbol}_{strategy}'    
            yPath= a_utils.addDir(company.img_path, company.symbol)
            yPath= a_utils.FilePath(Path=yPath, FileNm=filenm, TimeString='', Suffix='')
            plot.savefig(yPath) 






 wrapper241|10: (private) matplotlib data path: C:\Users\bt\anaconda3\lib\site-packages\matplotlib\mpl-data [__init__.py04/22:10:06]


 wrapper241|DEBUG: (private) matplotlib data path: C:\Users\bt\anaconda3\lib\site-packages\matplotlib\mpl-data [__init__.py04/22:10:06]


 wrapper241|10: matplotlib data path: C:\Users\bt\anaconda3\lib\site-packages\matplotlib\mpl-data [__init__.py04/22:10:06]


 wrapper241|DEBUG: matplotlib data path: C:\Users\bt\anaconda3\lib\site-packages\matplotlib\mpl-data [__init__.py04/22:10:06]


 wrapper241|10: CONFIGDIR=C:\Users\bt\.matplotlib [__init__.py04/22:10:06]


...

 <module>1436|DEBUG: Using fontManager instance from C:\Users\bt\.matplotlib\fontlist-v330.json [font_manager.py04/22:10:06]


 switch_backend289|10: Loaded backend module://ipykernel.pylab.backend_inline version unknown. [pyplot.py04/22:10:06]


 switch_backend289|DEBUG: Loaded backend module://ipykernel.pylab.backend_inline version unknown. [pyplot.py04/22:10:06]


 switch_backend289|10: Loaded backend module://ipykernel.pylab.backend_inline version unknown. [pyplot.py04/22:10:06]


 switch_backend289|DEBUG: Loaded backend module://ipykernel.pylab.backend_inline version unknown. [pyplot.py04/22:10:06]


In [11]:
# test code

from unittest import TestCase
import yfinance as yf
import pandas as pd
#wait:from yTA_Cal import set_technical_indicators, Company
#wait:from yTA_Plot import TA_chart_plotter


## for a TA function 
## tadf=add_all_ta_features(df1, open='open', high='high', low='low', close='close', volume='volume', fillna=True) 
config={}
yGoog=Company('GOOG')
yGoog.prices=  yf.Ticker(yGoog.symbol).history(period='1y')['Close']

#print('----------GooG')
#print(yGoog.prices)
#yGoog.prices=yDSTDA  #w yf.Ticker(yGoog.symbol).history(period='1y')['Close']


yTestCo=Company('TEST_Co')
###############################################################
DF2=pd.read_csv(r'C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\historical_data\a_Debug\GOOG_TDA_raw.csv')
DF2['Date']= pd.to_datetime(DF2['datetime'], unit='ms')  # datetime is timestamp /Epoch, Date is py datetime obj  
#DF2.rename(columns={'close':'Close'})
yTestCo.prices= pd.Series(DF2['close'].values, index=DF2['Date']) #price is PD series, with datetimeindex obj 

# yTestCo.name='Close'   #give PD series a name, not to any effect

###############################################################


#select between two different sources 
# from yahoo with pd ds with column "Close" and datetime item as index 
yTestCo=yGoog

set_technical_indicators(config, yTestCo)

#print(yTestCo.technical_indicators)
#print (yGoog.technical_indicators)

tacp=TechnicalIndicatorsChartPlotter() 
tacp.plot_macd(yTestCo)
tacp.plot_rsi(yTestCo)
tacp.plot_bollinger_bands(yTestCo)
tacp.plot_sma(yTestCo) 

#print('----------- TestCo')
#print("testco.prices type=", type(yTestCo.prices))
#print("testco.prices index type", type(yTestCo.prices.index))
#print("testco.prices index type", yTestCo.prices.dtypes)


#print('--------------')
#print ('DF2 type=', type(DF2))
#print("index type", type(DF2.index))
# print (' testo.tech ind =', DF2)


#print('--------------')
#print ('testco technical indicator type=', type(yTestCo.technical_indicators))
#print("testco.tech ind index type", type(yTestCo.technical_indicators.index))
#print (' testo.tech ind =', yTestCo.technical_indicators)

 _new_conn939|10: Starting new HTTPS connection (1): query1.finance.yahoo.com:443 [connectionpool.py04/22:10:06]


 _new_conn939|DEBUG: Starting new HTTPS connection (1): query1.finance.yahoo.com:443 [connectionpool.py04/22:10:06]


 _make_request433|10: https://query1.finance.yahoo.com:443 "GET /v8/finance/chart/GOOG?range=1y&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None [connectionpool.py04/22:10:06]


 _make_request433|DEBUG: https://query1.finance.yahoo.com:443 "GET /v8/finance/chart/GOOG?range=1y&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None [connectionpool.py04/22:10:06]


 _init_num_threads141|20: NumExpr defaulting to 8 threads. [utils.py04/22:10:06]


...

 _findfont_cached1346|10: findfont: score(<Font 'DengXian' (Deng.ttf) normal normal 400 normal>) = 10.05 [font_manager.py04/22:10:06]


 _findfont_cached1346|DEBUG: findfont: score(<Font 'DengXian' (Deng.ttf) normal normal 400 normal>) = 10.05 [font_manager.py04/22:10:06]


 _findfont_cached1346|10: findfont: score(<Font 'David CLM' (DavidCLM-BoldItalic.ttf) italic normal 700 normal>) = 11.335 [font_manager.py04/22:10:06]


 _findfont_cached1346|DEBUG: findfont: score(<Font 'David CLM' (DavidCLM-BoldItalic.ttf) italic normal 700 normal>) = 11.335 [font_manager.py04/22:10:06]


 _findfont_cached1346|10: findfont: score(<Font 'Microsoft New Tai Lue' (ntailu.ttf) normal normal 400 normal>) = 10.05 [font_manager.py04/22:10:06]
